# Imports

In [516]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import soundfile as sf
from scipy.fft import rfft,rfftfreq
import librosa
import librosa.display
import IPython.display as ipd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler    

sns.set()
warnings.filterwarnings('ignore')

# Model

### define features extraction function

In [517]:
def mfcc_feature_extractor(audio,sampleRate):
    mfccsFeatures = librosa.feature.mfcc(y=audio, sr=sampleRate, n_mfcc=40)
    mfccsScaledFeatures = np.mean(mfccsFeatures.T,axis=0)
    
    return mfccsScaledFeatures

In [518]:
def contrast_feature_extractor(audio,sampleRate):
    stft = np.abs(librosa.stft(audio))
    contrast = librosa.feature.spectral_contrast(S=stft, sr=sampleRate) 
    contrastScaled = np.mean(contrast.T,axis=0)
    
    return contrastScaled

In [519]:
def tonnetz_feature_extractor(audio,sampleRate):
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(audio), sr=sampleRate)
    tonnetzScaled = np.mean(tonnetz.T,axis=0)
    
    return tonnetzScaled

In [520]:
def centroid_feature_extractor(audio,sampleRate):
    centroid = librosa.feature.spectral_centroid(y=audio,sr=sampleRate) 
    centroidScaled = np.mean(centroid.T,axis=0)
    
    return centroidScaled

In [521]:
def features_extractor(file):
    features=[]
    audio, sampleRate = librosa.load(file, res_type='kaiser_fast') 
    mfcc=mfcc_feature_extractor(audio,sampleRate)
    contrast = contrast_feature_extractor(audio,sampleRate)
    tonnetz = tonnetz_feature_extractor(audio,sampleRate)
    centroid = centroid_feature_extractor(audio,sampleRate)
    features.append([mfcc,contrast,tonnetz,centroid])
    features[0] = np.concatenate((features[0][0],features[0][1],features[0][2],features[0][3]))
    return features

### define add features function

In [522]:
def add_features(extractedFeatures,dirPath,label):
    for file in os.listdir(dirPath):
        filePath = dirPath+'/'+file
        finalClassLabels=label
        audio, sampleRate = librosa.load(filePath, res_type='kaiser_fast') 
        mfcc=mfcc_feature_extractor(audio,sampleRate)
        contrast = contrast_feature_extractor(audio,sampleRate)
        tonnetz = tonnetz_feature_extractor(audio,sampleRate)
        centroid = centroid_feature_extractor(audio,sampleRate)
        extractedFeatures.append([mfcc,contrast,tonnetz,centroid,finalClassLabels])

### creating dictionary with dirPath and label

In [523]:
dict = {"anwar":"C:/Users/DELL/General/DSP Tasks/DSP_Task3/processing/anwar","aya":"C:/Users/DELL/General/DSP Tasks/DSP_Task3/processing/aya","ehab":"C:/Users/DELL/General/DSP Tasks/DSP_Task3/processing/ehab","zeyad":"C:/Users/DELL/General/DSP Tasks/DSP_Task3/processing/zeyad"}

### creating data frame from our data

In [524]:
extractedFeatures = []
for i in dict.keys():
    add_features(extractedFeatures,dict[i],i)

In [525]:
data=pd.DataFrame(extractedFeatures,columns=['mfcc','contrast','tonnetz','centroid','class'])
data.head()

,mfcc,contrast,tonnetz,centroid,class
0,"[-358.41626, 144.1046, -4.3864827, 17.072119, ...","[18.94650314889134, 16.090433605334066, 20.062...","[0.00294895926736136, 0.022882545163613432, 0....",[1221.7536077014136],anwar
1,"[-365.32745, 157.33589, 5.4608107, 15.8209915,...","[17.223154832056608, 16.212961272572127, 19.72...","[-0.010291890720739065, -0.015440910476654168,...",[1109.7785650830913],anwar
2,"[-317.8828, 143.54901, 3.7499225, 20.347023, 1...","[20.721853281004684, 18.815679529049962, 21.03...","[-0.014134320920242936, 0.017042053203192842, ...",[1179.2000128079796],anwar
3,"[-318.0163, 147.84286, -4.8235664, 18.210249, ...","[19.802830621137208, 18.10443040515306, 18.432...","[0.015136352665561117, 0.006968675769380706, -...",[1203.9084224676571],anwar
4,"[-306.87152, 142.04994, -7.6469383, 14.261601,...","[20.37519092151916, 18.49057983216712, 20.5218...","[0.02325556532372118, 0.004023271866620333, -0...",[1238.442289333621],anwar


### splitting the data into features and target

In [526]:
features=data.iloc[:,0:-1]
target=data['class']
features.head()

,mfcc,contrast,tonnetz,centroid
0,"[-358.41626, 144.1046, -4.3864827, 17.072119, ...","[18.94650314889134, 16.090433605334066, 20.062...","[0.00294895926736136, 0.022882545163613432, 0....",[1221.7536077014136]
1,"[-365.32745, 157.33589, 5.4608107, 15.8209915,...","[17.223154832056608, 16.212961272572127, 19.72...","[-0.010291890720739065, -0.015440910476654168,...",[1109.7785650830913]
2,"[-317.8828, 143.54901, 3.7499225, 20.347023, 1...","[20.721853281004684, 18.815679529049962, 21.03...","[-0.014134320920242936, 0.017042053203192842, ...",[1179.2000128079796]
3,"[-318.0163, 147.84286, -4.8235664, 18.210249, ...","[19.802830621137208, 18.10443040515306, 18.432...","[0.015136352665561117, 0.006968675769380706, -...",[1203.9084224676571]
4,"[-306.87152, 142.04994, -7.6469383, 14.261601,...","[20.37519092151916, 18.49057983216712, 20.5218...","[0.02325556532372118, 0.004023271866620333, -0...",[1238.442289333621]


In [527]:
mfcc=np.array(features['mfcc'].tolist())
contrast = np.array(features['contrast'].tolist())
tonnetz = np.array(features['tonnetz'].tolist())
centroid = np.array(features['centroid'].tolist())

In [528]:
features=[]
for i in range(len(mfcc)):
    features.append([mfcc[i],contrast[i],tonnetz[i],centroid[i]])
data=pd.DataFrame(features)
features = data.values.tolist()
for i in range(len(features)):
    features[i] = np.concatenate((features[i][0],features[i][1],features[i][2],features[i][3]))
features

[array([-3.58416260e+02,  1.44104599e+02, -4.38648272e+00,  1.70721188e+01,
         3.25647855e+00,  4.08728456e+00,  1.23469849e+01,  8.16565418e+00,
        -1.38202314e+01, -6.19025946e+00, -4.51332855e+00, -5.54141998e+00,
        -6.35425329e+00,  8.28024298e-02,  7.61330485e-01,  9.41562748e+00,
        -5.42610645e+00,  1.26269121e+01,  1.29096913e+00, -4.87843305e-01,
        -1.34291992e-01,  5.58941936e+00, -1.01491392e+00, -6.22229242e+00,
        -2.85267496e+00, -3.35822034e+00, -6.63623905e+00,  1.09199071e+00,
        -2.26326370e+00, -4.53420544e+00, -2.61406398e+00,  2.17159438e+00,
        -1.45526826e+00,  1.21795309e+00, -7.05423784e+00,  3.72012138e-01,
        -2.05930686e+00,  2.99561769e-01, -3.88594776e-01, -4.26979971e+00,
         1.89465031e+01,  1.60904336e+01,  2.00620759e+01,  1.99617040e+01,
         1.97411258e+01,  2.00444648e+01,  4.63433879e+01,  2.94895927e-03,
         2.28825452e-02,  5.24837532e-02,  2.82412638e-02,  1.31465592e-02,
         8.6

### encoding the target

In [529]:
encoder = LabelEncoder()
target = encoder.fit_transform(target)

### splitting the data into train and test

In [530]:
xTrain, xTest, yTrain, yTest = train_test_split(features, target, test_size=0.7, random_state=0)

### creating SVM model

In [531]:
classifier= DecisionTreeClassifier(criterion='entropy', random_state=0)  
classifier.fit(features,target)

DecisionTreeClassifier(criterion='entropy', random_state=0)

### predicting xTest

In [532]:
preds = classifier.predict(xTest)

In [533]:
def prediction(pred):
    if pred==0:
        print("anwar")
    elif pred==1:
        print("aya")
    elif pred==2:
        print("ehab")
    elif pred==3:
        print("zeyad")

### making classification report to see the result of the predicton 

In [534]:
print(classification_report(yTest,preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       1.00      1.00      1.00        11
           2       1.00      1.00      1.00        11
           3       1.00      1.00      1.00        11

    accuracy                           1.00        42
   macro avg       1.00      1.00      1.00        42
weighted avg       1.00      1.00      1.00        42



In [535]:
testFeatures = features_extractor("C:/Users/DELL/General/DSP Tasks/DSP_Task3/processing/zeyad/zeyad5.wav")
pred = classifier.predict(testFeatures)
prediction(pred)


zeyad


### creating pickle

In [536]:
pickle_out = open("model.pkl", "wb")
pickle.dump(classifier, pickle_out)
pickle_out.close()